In [2]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('..')

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns

from src.representation import get_representation, get_representation_from_series
from src.config import mem
from src.utils import get_fps_offset

N_JOBS = 12
RANDOM_SEED = 42

# from rdkit import RDLogger
# RDLogger.DisableLog('rdApp.*')

X_train = pd.read_csv('../data/processed/X_train.csv', index_col=0)
y_train = pd.read_csv('../data/processed/y_train.csv', index_col=0)
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 961 entries, DTXSID9048512 to DTXSID3040352
Columns: 2790 entries, 0 to md_Zagreb2
dtypes: float64(2790)
memory usage: 20.5+ MB


In [4]:
FPS_OFFSET = get_fps_offset(X_train.columns)
FPS_OFFSET

2363

In [50]:
import optuna

class FeatureSelectionOptuna:
    """
    This class implements feature selection using Optuna optimization framework.

    Parameters:

    - model (object): The predictive model to evaluate; this should be any object that implements fit() and predict() methods.
    - loss_fn (function): The loss function to use for evaluating the model performance. This function should take the true labels and the
                          predictions as inputs and return a loss value.
    - features (list of str): A list containing the names of all possible features that can be selected for the model.
    - X (DataFrame): The complete set of feature data (pandas DataFrame) from which subsets will be selected for training the model.
    - y (Series): The target variable associated with the X data (pandas Series).
    - splits (list of tuples): A list of tuples where each tuple contains two elements, the train indices and the validation indices.
    - penalty (float, optional): A factor used to penalize the objective function based on the number of features used.
    """

    def __init__(self,
                 model,
                 loss_fn,
                 features,
                 X,
                 y,
                 splits,                 
                 penalty=0,
                 ):

        self.model = model
        self.loss_fn = loss_fn
        self.features = features
        self.X = X
        self.y = y
        self.splits = splits
        self.penalty = penalty

    def __call__(self,
                 trial: optuna.trial.Trial):

        # Select True / False for each feature
        selected_features = [trial.suggest_categorical(name, [True, False]) for name in self.features]

        # List with names of selected features
        selected_feature_names = [name for name, selected in zip(self.features, selected_features) if selected]

        # Optional: adds a penalty for the amount of features used
        n_used = len(selected_feature_names)
        total_penalty = n_used * self.penalty

        loss = 0

        for split in self.splits:
          train_idx = split[0]
          valid_idx = split[1]

          X_train = self.X.iloc[train_idx].copy()
          y_train = self.y.iloc[train_idx].copy()
          X_valid = self.X.iloc[valid_idx].copy()
          y_valid = self.y.iloc[valid_idx].copy()

          X_train_selected = X_train[selected_feature_names].copy()
          X_valid_selected = X_valid[selected_feature_names].copy()

          # Train model, get predictions and accumulate loss
          self.model.fit(X_train_selected, y_train)
          pred = self.model.predict(X_valid_selected)

          loss += self.loss_fn(y_valid, pred)

        # Take the average loss across all splits
        loss /= len(self.splits)

        # Add the penalty to the loss
        loss += total_penalty
        
        return loss

In [51]:
from sklearn.metrics import root_mean_squared_error as rmse
from sklearn.model_selection import KFold
from optuna.samplers import TPESampler
import xgboost as xgb

mask = X_train.columns.str.contains(r'[rd_|md_]')
features = X_train.columns[mask]
print('Features:', len(features))

kfold = KFold(n_splits=10, shuffle=True, random_state=RANDOM_SEED)
splits = list(kfold.split(X_train))

model = xgb.XGBRegressor(random_state=RANDOM_SEED, n_jobs=N_JOBS, verbosity=0)

sampler = TPESampler(seed=RANDOM_SEED)
study = optuna.create_study(direction="minimize",sampler=sampler)

# We first try the model using all features
default_features = {ft: True for ft in features}
study.enqueue_trial(default_features)

Features: 427


In [53]:
optuna.logging.set_verbosity(optuna.logging.WARN)

study.optimize(FeatureSelectionOptuna(
                         model=model,
                         loss_fn=rmse,
                         features=features,
                         X=X_train,
                         y=y_train,
                         splits=splits,
                         # penalty = 1e-4,  
                         ), n_trials=100, show_progress_bar=True)

  0%|          | 0/100 [00:00<?, ?it/s]

[W 2024-08-26 12:46:17,305] Trial 4 failed with parameters: {'rd_BalabanJ': False, 'rd_Chi2v': True, 'rd_EState_VSA1': False, 'rd_EState_VSA11': False, 'rd_EState_VSA3': False, 'rd_EState_VSA4': False, 'rd_EState_VSA5': False, 'rd_EState_VSA6': True, 'rd_EState_VSA7': False, 'rd_EState_VSA8': False, 'rd_FpDensityMorgan1': True, 'rd_FpDensityMorgan2': False, 'rd_FpDensityMorgan3': False, 'rd_FractionCSP3': False, 'rd_HallKierAlpha': True, 'rd_Ipc': True, 'rd_Kappa2': False, 'rd_Kappa3': False, 'rd_MaxEStateIndex': True, 'rd_MinAbsEStateIndex': True, 'rd_MinEStateIndex': True, 'rd_NumAliphaticCarbocycles': False, 'rd_NumHDonors': False, 'rd_NumRadicalElectrons': True, 'rd_NumSaturatedCarbocycles': True, 'rd_PEOE_VSA10': False, 'rd_PEOE_VSA14': True, 'rd_PEOE_VSA2': True, 'rd_PEOE_VSA3': True, 'rd_PEOE_VSA4': False, 'rd_PEOE_VSA5': False, 'rd_RingCount': False, 'rd_SMR_VSA1': False, 'rd_SMR_VSA10': True, 'rd_SMR_VSA5': False, 'rd_SMR_VSA6': False, 'rd_SMR_VSA7': True, 'rd_SMR_VSA9': True,

KeyboardInterrupt: 

In [81]:
opt_features = [k for k,v in study.trials[73].params.items() if v]
len(opt_features)

209

In [83]:
opt_features

['rd_Chi2v',
 'rd_EState_VSA3',
 'rd_EState_VSA6',
 'rd_EState_VSA8',
 'rd_HallKierAlpha',
 'rd_Ipc',
 'rd_Kappa2',
 'rd_MaxEStateIndex',
 'rd_MinAbsEStateIndex',
 'rd_NumAliphaticCarbocycles',
 'rd_NumHDonors',
 'rd_NumRadicalElectrons',
 'rd_PEOE_VSA14',
 'rd_PEOE_VSA2',
 'rd_PEOE_VSA3',
 'rd_PEOE_VSA4',
 'rd_PEOE_VSA5',
 'rd_SMR_VSA10',
 'rd_SMR_VSA6',
 'rd_SlogP_VSA10',
 'rd_SlogP_VSA7',
 'rd_VSA_EState3',
 'rd_VSA_EState4',
 'rd_VSA_EState5',
 'rd_VSA_EState9',
 'rd_fr_ArN',
 'rd_fr_Ar_N',
 'rd_fr_COO2',
 'rd_fr_C_O',
 'rd_fr_C_O_noCOO',
 'rd_fr_Imine',
 'rd_fr_NH1',
 'rd_fr_alkyl_halide',
 'rd_fr_allylic_oxid',
 'rd_fr_ether',
 'rd_fr_furan',
 'rd_fr_guanido',
 'rd_fr_hdrzone',
 'rd_fr_ketone',
 'rd_fr_nitro_arom_nonortho',
 'rd_fr_nitroso',
 'rd_fr_oxazole',
 'rd_fr_phos_ester',
 'rd_fr_piperdine',
 'rd_fr_piperzine',
 'rd_fr_priamide',
 'rd_fr_sulfonamd',
 'rd_fr_sulfone',
 'rd_fr_thiazole',
 'rd_fr_thiophene',
 'rd_fr_unbrch_alkane',
 'md_nAcid',
 'md_nBase',
 'md_nHetero',
 '